In [ ]:
import gc
import os

from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# NAMES = ['LL','LP','RP','RR']

# FEATS = [['Fp1','F7','T3','T5','O1'],
#          ['Fp1','F3','C3','P3','O1'],
#          ['Fp2','F8','T4','T6','O2'],
#          ['Fp2','F4','C4','P4','O2']]

train = pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/train.csv')

In [ ]:
train

### Смотрю, где прячутся наны

In [ ]:
from collections import defaultdict

dct = defaultdict(list)

for eeg_id in tqdm(train.eeg_id.unique()):
    eeg = pd.read_parquet(f'/kaggle/input/hms-harmful-brain-activity-classification/train_eegs/{eeg_id}.parquet')
    sl = train[train.eeg_id == eeg_id].eeg_label_offset_seconds
    for offset in sl:
        pre_target_part = eeg.iloc[int(offset)*200: (50 + int(offset)) * 200 - 30 * 200, :].copy().reset_index(drop=True)
        target_part = eeg.iloc[int(offset)*200 + 20*200: (50 + int(offset)) * 200 - 20 * 200, :].copy().reset_index(drop=True)
        post_target_part = eeg.iloc[int(offset)*200 + 30*200: (50 + int(offset)) * 200, :].copy().reset_index(drop=True)
        na = target_part['Fp1'].isna()
        pre_na = pre_target_part['Fp1'].isna()
        post_na = post_target_part['Fp1'].isna()

        if na.any() or pre_na.any() or post_na.any():
            dct[f'{eeg_id}_{offset}'].append([pre_na.sum(), na.sum(), post_na.sum()])

In [ ]:
def get_nans_counts(dct, add=''):
    for i, name in zip([0,1,2], [f'_pre_target_slice{add}', f'_traget_slice{add}', f'_post_target_slice{add}']):
        lst1 = []
        lst2 = []
        for key in dct.keys():
            if dct[key][0][i] != 0:
                eeg_id, off = key.split('_')
                lst1.append(eeg_id)
                lst2.append(off)
    
        pd.DataFrame(np.array([lst1, lst2]).T, columns=['eeg_id', 'eeg_label_offset_seconds']).to_csv(f'any_nans_in_input{name}.csv', index=False)
        
    lst1 = []
    lst2 = []
    for key in dct.keys():
        eeg_id, off = key.split('_')
        lst1.append(eeg_id)
        lst2.append(off)

    pd.DataFrame(np.array([lst1, lst2]).T, columns=['eeg_id', 'eeg_label_offset_seconds']).to_csv(f'any_nans_in_input{add}.csv', index=False)

In [ ]:
get_nans_counts(dct)

In [ ]:
from collections import defaultdict

dct_int = defaultdict(list)

for eeg_id in tqdm(train.eeg_id.unique()):
    eeg = pd.read_parquet(f'/kaggle/input/hms-harmful-brain-activity-classification/train_eegs/{eeg_id}.parquet')['Fp1']
    #Как я делал усреднение
    eeg[eeg.isnull()] = (eeg.shift(-1) + eeg.shift(1))/2
    sl = train[train.eeg_id == eeg_id].eeg_label_offset_seconds
    for offset in sl:
        pre_target_part = eeg.iloc[int(offset)*200: (50 + int(offset)) * 200 - 30 * 200].copy().reset_index(drop=True)
        target_part = eeg.iloc[int(offset)*200 + 20*200: (50 + int(offset)) * 200 - 20 * 200].copy().reset_index(drop=True)
        post_target_part = eeg.iloc[int(offset)*200 + 30*200: (50 + int(offset)) * 200].copy().reset_index(drop=True)
        na = target_part.isna()
        pre_na = pre_target_part.isna()
        post_na = post_target_part.isna()

        if na.any() or pre_na.any() or post_na.any():
            dct_int[f'{eeg_id}_{offset}'].append([pre_na.sum(), na.sum(), post_na.sum()])

In [ ]:
get_nans_counts(dct_int, add='_intrapolate_1')

### Заканчиваю смотреть, где прячутся наны

In [ ]:
# def check(list):
#     return all(i == list[0] for i in list)
# for key in dct.keys():
#     if not check(dct[key]):
#         print(key)

In [ ]:
one_nans = []
more_nans = []
for key in dct.keys():
    if dct[key][0] > 1:
        more_nans.append(key)
    else:
        one_nans.append(key)

In [ ]:
len(one_nans), len(more_nans)

In [ ]:
[dct[key][0] for key in more_nans]

### STFT

In [ ]:
# import librosa

# def spectrogram_from_eeg(parquet_path, display=False):
#     eeg = pd.read_parquet(parquet_path)
#     specs = {}
#     for col in eeg.columns:
#         flag = False
# #         mel_spec = librosa.feature.melspectrogram(y=eeg[col].values, sr=200, hop_length=len(eeg[col])//256, 
# #                   n_fft=1024, n_mels=128, fmin=0, fmax=20, win_length=128)
#         if eeg[col].isna().any():
#             stft_spec = librosa.stft(y=eeg[col].fillna(0).values, hop_length=len(eeg[col])//256, n_fft=128, win_length=128)
#         else:
#             continue
#         stft_spec = np.log1p(np.abs(np.real(stft_spec)))
#         specs[col] = stft_spec
        
    return specs

In [ ]:
# def spectrogram_from_eeg(parquet_path, display=False):
#     eeg = pd.read_parquet(parquet_path)
#     specs = {}
#     for col in eeg.columns:
#         flag = False
# #         mel_spec = librosa.feature.melspectrogram(y=eeg[col].values, sr=200, hop_length=len(eeg[col])//256, 
# #                   n_fft=1024, n_mels=128, fmin=0, fmax=20, win_length=128)
#         stft_spec = librosa.stft(y=eeg[col].fillna(0).values, hop_length=len(eeg[col])//256, 
#                 n_fft=128, win_length=128)
#         stft_spec = np.log1p(np.abs(np.real(stft_spec)))
#         specs[col] = stft_spec
        
#     return specs

### Wavelet

In [ ]:
import pywt
dt = 1/200
frequencies = pywt.scale2frequency('mexh', np.arange(1.5, 100, 0.5)) / dt

In [ ]:
def normalize8(img):
    mn = img.min()
    mx = img.max()
    mx -= mn
    img = ((img - mn)/mx) * 255
    return img.astype(np.uint8)

# def spectrogram_from_eeg(parquet_path, col):
#     eeg = pd.read_parquet(parquet_path)[col].fillna(0).values
#     widths = 1
#     wave_spec = pywt.cwt(eeg, widths, 'mexh')
#     return wave_spec.astype(np.float32)

In [ ]:
%%time
PATH = '/kaggle/input/hms-harmful-brain-activity-classification/train_eegs/'
EEG_IDS = train.eeg_id.unique()
all_eegs = {}

for eeg_id in tqdm(EEG_IDS):
    specs = spectrogram_from_eeg(f'{PATH}{eeg_id}.parquet')
    if specs:
        all_eegs[eeg_id] = specs

In [ ]:
plt.imshow(all_eegs[4045810693]['Fp1'])

In [ ]:
all_eegs.keys()

In [ ]:
# %%time
# PATH = '/kaggle/input/hms-harmful-brain-activity-classification/train_eegs/'
# EEG_IDS = train.eeg_id.unique()
# all_eegs = {}
# c = 0
# j = len(EEG_IDS) // 2

# for el in ['Fp1']:# 'F7','T3','T5','O1','Fp1','F3','C3','P3','O1','Fp2','F8','T4','T6','O2','Fp2','F4','C4','P4','O2']:
#     for eeg_id in tqdm(EEG_IDS):
#         specs = spectrogram_from_eeg(f'{PATH}{eeg_id}.parquet', el)
# #         all_eegs[eeg_id] = normalize8(specs)
#         all_eegs[eeg_id] = specs
#     np.save(f'Wavelet_eeg_specs_{el}',all_eegs)
#     del all_eegs
#     gc.collect()
#     all_eegs = {}